# Predicciones de Riesgos de Accidentes de Tránsido con Machine Learning

## Objetivo General
Aplicar las técnicas de Machine Learning para realizar predicciones sobre el fenómeno de siniestros de tránsito utilizando el conjunto de datos de la AMT de Ecuador con fecha de corte 2024-x-x.
## Objetivo Específico:
- Utilizar aprendizaje supervisado que permita predecir, dados unas condiciones de entrada, el nivel de gravedad del accidente. Para esto se considera la **Pirámide de Bird**, la cual se adapta según la relación de fallecidos y lesionados en el siniestro.
- Utilizar aprendizaje supervisado para predecir la cantidad de lesionados y fallecidos según condiciones de entrada. Para esto se puede partir del dataset original.
- Utilizar aprendizaje supervisado para obtener grupos de siniestralidad en función de las características del accidente

In [1]:
import pandas as pd
import os
dataset_path = os.path.join(os.getcwd(), 'data', 'dataset_original.xlsx')
dataset = pd.read_excel(dataset_path)
dataset.shape

(165017, 60)

In [2]:
pd.set_option('display.max_columns', None)
pd.options.display.max_rows = None

In [3]:
dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 165017 entries, 0 to 165016
Data columns (total 60 columns):
 #   Column                         Non-Null Count   Dtype  
---  ------                         --------------   -----  
 0   NUMERO                         165017 non-null  int64  
 1   ANIO                           165017 non-null  int64  
 2   SINIESTROS                     165017 non-null  object 
 3   LESIONADOS                     165017 non-null  int64  
 4   FALLECIDOS                     165017 non-null  int64  
 5   ENTE_DE_CONTROL                165017 non-null  object 
 6   LATITUD_Y                      165017 non-null  float64
 7   LONGITUD_X                     165017 non-null  float64
 8   DPA_1                          165017 non-null  int64  
 9   PROVINCIA                      165017 non-null  object 
 10  DPA_2                          165017 non-null  int64  
 11  CANTON                         165017 non-null  object 
 12  DPA_3                         

In [4]:
dataset_cl1 = dataset.dropna()
dataset_cl1.shape

(164992, 60)

## Valores únicos contenidos en el dataset

Se revisa cuál es el contenido de valores únicos en el dataset en el caso de que la columna sea de tipo object

In [14]:
for column in ['FERIADO', 'CODIGO_CAUSA', 'CAUSA_PROBABLE', 'TIPO_DE_SINIESTRO', 'TIPO_DE_VEHICULO_1','SERVICIO_1', 'PARTICIPANTE_1', 'CASCO_1', 'CINTURON_1', 'GRUPO ETARIO', 'SEXO_1']:
    if dataset_cl1[column].dtype == "object":
        print(f"{column} tiene {len(set(dataset_cl1[column]))} valores únicos y {dataset_cl1[column].isna().sum()} valores nulos:\n{set(dataset_cl1[column])}")

FERIADO tiene 2 valores únicos y 0 valores nulos:
{'SI', 'NO'}
CODIGO_CAUSA tiene 27 valores únicos y 0 valores nulos:
{'C02', 'C18', 'C15', 'C12', 'C27', 'C05', 'C22', 'C23', 'C09', 'C01', 'C06', 'C14', 'C11', 'C26', 'C08', 'C21', 'C10', 'C19', 'C07', 'C03', 'C28', 'C04', 'C20', 'C17', 'C25', 'C24', 'C16'}
CAUSA_PROBABLE tiene 27 valores únicos y 0 valores nulos:
{'DEJAR O RECOGER PASAJEROS EN LUGARES NO PERMITIDOS.', 'DISPOSITIVO REGULADOR DE TRANSITO EN MAL ESTADO DE FUNCIONAMIENTO (SEMAFORO).', 'NO MANTENER LA DISTANCIA PRUDENCIAL CON RESPECTO AL VEHICULO QUE LE ANTECEDE.', 'NO TRANSITAR POR LAS ACERAS O ZONAS DE SEGURIDAD DESTINADAS PARA EL EFECTO.', 'BAJARSE O SUBIRSE DE VEHICULOS EN MOVIMIENTO SIN TOMAR LAS PRECAUCIONES DEBIDAS.', 'ADELANTAR O REBASAR A OTRO VEHICULO EN MOVIMIENTO EN ZONAS O SITIOS PELIGROSOS TALES COMO: CURVAS, PUENTES, TUNELES, PENDIENTES, ETC.', 'FALLA MECANICA EN LOS SISTEMAS Y/O NEUMATICOS (SISTEMA DE FRENOS, DIRECCION, ELECTRONICO O MECANICO).', 'NO CEDER 

In [16]:
# dataset_cl1['GRUPO ETARIO'].value_counts()

In [17]:
# dataset_cl1.SEXO_1.value_counts()

In [11]:
# dataset_cl1.sort_values(by='SINIESTROS', ascending=True).head(10)
# dataset_cl1[dataset_cl1.duplicated(subset='SINIESTROS', keep=False)]
dataset_cl1.sample(10)

,NUMERO,ANIO,SINIESTROS,LESIONADOS,FALLECIDOS,ENTE_DE_CONTROL,LATITUD_Y,LONGITUD_X,DPA_1,PROVINCIA,DPA_2,CANTON,DPA_3,PARROQUIA,DIRECCION,ZONA_PLANIFICACION,ZONA,ID_DE_LA_VIA,NOMBRE_DE_LA_VIA,UBICACION_DE_LA_VIA,JERARQUIA_DE_LA_VIA,FECHA,HORA,PERIODO_1,PERIODO_2,DIA_1,DIA_2,MES_1,MES_2,FERIADO,CODIGO_CAUSA,CAUSA_PROBABLE,TIPO_DE_SINIESTRO,TIPO_DE_VEHICULO_1,SERVICIO_1,AUTOMOVIL,BICICLETA,BUS,CAMION,CAMIONETA,EMERGENCIAS,ESPECIAL,FURGONETA,MOTOCICLETA,NO_IDENTIFICADO,SCOOTER_ELECTRICO,TRICIMOTO,VEHICULO_DEPORTIVO_UTILITARIO,SUMA_DE_VEHICULOS,TIPO_ID_1,EDAD_1,SEXO_1,CONDICION_1,PARTICIPANTE_1,CASCO_1,CINTURON_1,DPA_PROV,DPA_CANT,DPA_PARR,GRUPO ETARIO
91584,8236,2017,MCU08236042017,2,0,"EMPRESA PUBLICA DE MOVILIDAD, TRANSITO Y TRANS...",-2.908057,-79.057243,1,AZUAY,101,CUENCA,10151,BAÑOS (CUENCA),LOS MIGRANTES Y 25 DE MARZO,ZONA 6,URBANA,ND,ND,ND,ND,"sábado, 15 de abril de 2017",09:05:00,DE 09H00 A 09H59,9.0,SABADO,6.0,ABRIL,4.0,NO,C09,CONDUCIR VEHICULO SUPERANDO LOS LIMITES MAXIMO...,PERDIDA DE PISTA,CAMIONETA,GOBIERNOS SECCIONALES,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,LICENCIA,35.0,HOMBRE,LESIONADO,CONDUCTOR PRESENTE,NO,NO,1,101,10151,DE 30 A 39 AÑOS
149550,97984,2021,ATM01920022021,2,0,AGENCIA DE TRANSITO Y MOVILIDAD DE GUAYAQUIL -...,-2.186508,-79.954178,9,GUAYAS,901,GUAYAQUIL,90150,GUAYAQUIL,AV. PERIMETRAL,ZONA 8,URBANA,177,KM 19.4 VIA GUAYAQUIL / PROGRESO - GUAYAQUIL,E40,ARTERIAL,"miércoles, 3 de febrero de 2021",22:00:00,DE 22H00 A 22H59,22.0,MIERCOLES,3.0,FEBRERO,2.0,NO,C09,CONDUCIR VEHICULO SUPERANDO LOS LIMITES MAXIMO...,PERDIDA DE PISTA,VEHICULO DEPORTIVO UTILITARIO,PARTICULAR,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,2.0,LICENCIA,-1.0,HOMBRE,LESIONADO,CONDUCTOR PRESENTE,NO,SI,9,901,90150,NO IDENTIFICADO
78272,36408,2018,MAM07441042018,1,0,"DIRECCION DE TRANSITO, TRANSPORTE TERRESTRE Y ...",-1.217724,-78.611363,18,TUNGURAHUA,1801,AMBATO,180152,ATAHUALPA (CHISALATA),AV 12 DE SEPTIEMBRE Y DESTACAMENTO TWINZA,ZONA 3,RURAL,ND,ND,ND,ND,"jueves, 19 de abril de 2018",22:00:00,DE 22H00 A 22H59,22.0,JUEVES,4.0,ABRIL,4.0,NO,C25,NO CEDER EL DERECHO DE VIA O PREFERENCIA DE PA...,CHOQUE LATERAL,MOTOCICLETA,PARTICULAR,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,CEDULA,35.0,HOMBRE,LESIONADO,CONDUCTOR PRESENTE,NO,NO,18,1801,180152,DE 30 A 39 AÑOS
91672,7171,2017,PNE07171042017,2,0,POLICIA NACIONAL DEL ECUADOR,-3.254477,-79.957525,7,EL ORO,701,MACHALA,70150,MACHALA,AV. GUABO AV. AYACUCHO,ZONA 7,URBANA,ND,ND,ND,ND,"sábado, 1 de abril de 2017",19:30:00,DE 19H00 A 19H59,19.0,SABADO,6.0,ABRIL,4.0,NO,C12,NO GUARDAR LA DISTANCIA LATERAL MINIMA DE SEGU...,CHOQUE LATERAL,AUTOMOVIL,PUBLICO,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,2.0,CEDULA,40.0,HOMBRE,LESIONADO,CONDUCTOR PRESENTE,NO,NO,7,701,70150,DE 40 A 49 AÑOS
128978,116972,2021,CTE20908122021,0,0,COMISION DE TRANSITO DEL ECUADOR - CTE,-0.234083,-79.136833,23,SANTO DOMINGO DE LOS TSACHILAS,2301,SANTO DOMINGO,230150,SANTO DOMINGO DE LOS COLORADOS,RED VIAL ESTATAL E25 KM 35 SANTO DOMINGO LOS B...,ZONA 4,RURAL,126,LIMITE PROVINCIAL ENTRE SANTO DOMINGO Y PICHIN...,E25,ARTERIAL,"sábado, 25 de diciembre de 2021",06:30:00,DE 06H00 A 06H59,6.0,SABADO,6.0,DICIEMBRE,12.0,SI,C23,NO RESPETAR LAS SEÑALES REGLAMENTARIAS DE TRAN...,CHOQUE LATERAL,CAMION,PARTICULAR,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,LICENCIA,46.0,HOMBRE,ILESO,CONDUCTOR PRESENTE,NO,NO,23,2301,230150,DE 40 A 49 AÑOS
50215,122608,2022,MCU00018042022,0,0,"EMPRESA PUBLICA DE MOVILIDAD, TRANSITO Y TRANS...",-2.911663,-79.014009,1,AZUAY,101,CUENCA,10150,CUENCA,AV PRIMERO DE MAYO Y AVENIDA FELIPE II,ZONA 6,URBANA,ND,ND,ND,ND,"viernes, 1 de abril de 2022",23:55:00,DE 23H00 A 23H59,23.0,VIERNES,5.0,ABRIL,4.0,NO,C25,NO CEDER EL DERECHO DE VIA O PREFERENCIA DE PA...,CHOQUE LATERAL,VEHICULO DEPORTIVO UTILITARIO,PARTICULAR,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,2.0,LICENCIA,29.0,HOMBRE,ILESO,CONDUCTOR PRESENTE,NO,NO,1,101,10150,DE 20 A 29 AÑOS
108214,30885,2018,CTE01918